In [6]:
import nltk
nltk.download('rslp')
nltk.download('stopwords')
import re
import codecs
import unicodedata
from unicodedata import normalize

[nltk_data] Downloading package rslp to C:\Users\Uciolli
[nltk_data]     Betelli\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Uciolli
[nltk_data]     Betelli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
class NLP_Kit(object):
	#construtor da classe
	def __init__(self, stopwordsFile = None):
		'''Caso voce queira passar um arquivo de stopwords proprio,
		utilize-o durante a construcao do objeto NLP_kit com seu caminho
		absoluto'''
		self.stemmer = nltk.stem.RSLPStemmer()
		self.sent_tokenizer = None
		self.stopwords = None
		self.stopwordsFile = stopwordsFile

	def normaliza(self, texto):
		'''Retira acentos do texto e o retorna
		'''
		try:
			return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore')
		except:
			return unicodedata.normalize('NFKD', texto.decode('UTF-8')).encode('ASCII', 'ignore')
		raise Exception('Erro no formato da string. Utilize unicode utf-8')

	def separa_frases(texto):
		'''
		Recebe um texto. Retorna lista com as frases contidas no texto que possuem mais
		de 1 caracter
		'''
		#adiciona espacos entre pontuacoes, para facilitar o trabalho do sent_tokenizer.
		texto = re.sub('([.,!?()])', r' \1 ', texto)
		texto = re.sub('\s{2,}', ' ', texto)

		if self.sent_tokenizer is None:
			self.sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')
		sentencas =  self.sent_tokenizer.tokenize(texto)
		sentencas = [sentenca for sentenca in sentencas if len(sentenca) > 1]
		return sentencas

	def tokenize(self, frase):
		'''Recebe uma frase. Retorna lista de tokens (palavras) da frase
		que sao separadas por espaco ou virgula, travessao etc
		Retorna em Unicode e minusculo
		'''
		frase = frase.lower()
		if not isinstance(frase, str):
			frase = frase.decode('utf-8')
		return re.findall(r'\w+', frase,flags = re.UNICODE)

	def stemiza(palavra):
		'''Recebe uma palavra. Retorna seu stem
		'''
		return nltk.stemmer.stem(palavra)

	def remove_stop_words(self, texto):
		'''Retorna lista de palavras sem stopwords 
		'''
		if self.stopwords is None:
			if self.stopwordsFile is not None:
				stopwords = [word for word in codecs.open(stopwordsFile, 'r', 'utf-8')]
			else:
				stopwords = nltk.corpus.stopwords.words('portuguese')

			stopwords = [str( self.normaliza(item), "utf-8" ) for item in stopwords]

			palavras = self.tokenize(texto)
			
			#print type(stopwords[1])
			palavras_importantes = [palavra for palavra in palavras if palavra.lower() not in stopwords]
		return palavras_importantes

	def palavras_mais_frequentes(self, texto):
		'''retorna lista de tuplas <palavra, frequencia>
		'''
		palavras = self.remove_stop_words(texto)

		fd = nltk.FreqDist(palavras)
		frequencias = []
		for palavra in fd:
			frequencias.append( (palavra, fd[palavra]) )
		return frequencias

	def palavras_mais_frequentes2(self, texto): #com stopwords
		'''retorna lista de tuplas <palavra, frequencia>
		'''
		palavras = texto.split()

		fd = nltk.FreqDist(palavras)
		frequencias = []
		for palavra in fd:
			frequencias.append( (palavra, fd[palavra]) )
		return frequencias

	def remove_palavra_stop_words(self, texto):#retorna frase
		'''Retorna palavras sem stopwords 
		'''
		if self.stopwords is None:
			if self.stopwordsFile is not None:
				stopwords = [word for word in codecs.open(stopwordsFile, 'r', 'utf-8')]
			else:
				stopwords = nltk.corpus.stopwords.words('portuguese')

			stopwords = [str( self.normaliza(item), "utf-8" ) for item in stopwords]
			#print(stopwords)    #mostra quais sao os stopwords retirados
			palavras = texto.split(' ')

			frase_sem_sw= ''
			
			for palavra in palavras:
				if palavra not in stopwords:
					frase_sem_sw += palavra+' '
		return frase_sem_sw

	def remove_pontuacao(self, texto):#retorna frase
		string_normalizada = re.sub(':|;|\[|\]|\(|\)|,|\.|!|\?|\{|\}|\*|\"|-', ' ', texto.replace('\n',' '))	#retira pontuacao
		string_normalizada = string_normalizada.replace('//',' ')
		string_normalizada = string_normalizada.replace('\'','')
		return string_normalizada

In [8]:
#texto = "Teste De verificação :) de um Tweet teste!"
#frases = NLP_Kit()
#string_normalizada = frases.remove_pontuacao(texto)
#print(string_normalizada)
#string_normalizada = frases.normaliza(string_normalizada)   #retira acentos
#string_normalizada = str( string_normalizada, "utf-8" )
#print(string_normalizada)
#string_normalizada = frases.remove_palavra_stop_words(string_normalizada.lower())
#print(string_normalizada)
#string_normalizada = re.sub(' +', ' ', string_normalizada) #remove espacos em branco multiplos
#print(string_normalizada)
#string_normalizada = ((unicodedata.normalize('NFKD', string_normalizada)).encode('ASCII', 'ignore').lower())
#print(string_normalizada)
#string_normalizada = frases.palavras_mais_frequentes(string_normalizada)
#print(string_normalizada)

In [9]:
#import csv
#with open('tweets_ofensivos_sem_repeticao.csv', encoding="utf8") as f:
#    reader = csv.reader(f)
#    lista = list(reader)

In [10]:
import csv
import pandas as pd

def processa_texto_base(caminho_in, caminho_out):
    with open(caminho_in, encoding="utf8") as f:
        reader = csv.reader(f)
        lista = list(reader)
        
    lista_processada = []
    frases = NLP_Kit()
    for row in lista:
        texto_processando = frases.remove_pontuacao(row[0])
        texto_processando = frases.normaliza(texto_processando)
        texto_processando = str( texto_processando, "utf-8" )
        texto_processando = frases.remove_palavra_stop_words(texto_processando.lower())
        texto_processando = re.sub(' +', ' ', texto_processando)
        lista_processada.append(texto_processando)

    dict_lista = {'text': lista_processada}
    df = pd.DataFrame(dict_lista)
    df.to_csv(caminho_out, index=False)

In [12]:
processa_texto_base('csvs/pre-processado/tweets_odio_bolsonaro_texto.csv', 'csvs/processados/tweets_odio_bolsonaro_processado.csv')

In [ ]:
processa_texto_base('csvs/pre-processado/tweets_odio_lula_texto.csv', 'csvs/processados/tweets_odio_lula_processado.csv')